In [32]:
import pandas as pd
import numpy as np 

# URL DF processing

## data preprocessing

In [4]:
#load dataset
urlsdf = pd.read_csv('df_urls.csv')
urlsdf = urlsdf.drop(columns=['Unnamed: 0'], axis=1)
urlsdf['label'] = urlsdf['label'].replace(to_replace='no-clickbait', value=0)
urlsdf['label'] = urlsdf['label'].replace(to_replace='clickbait', value=1)

 
#process url column of url dataset
from ast import literal_eval
urlsdf['url'] = urlsdf['url'].tolist()
urlsdf['url'] = urlsdf['url'].apply(literal_eval)
urlsdf = urlsdf.explode(column='url')


In [35]:
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from time import sleep

titlesdf = pd.DataFrame(["title", "label"])


def findTitles(urllist, yn):
    label = yn
    titlelist = []
    labellist = []
    for url in urllist:
        session = requests.Session()
        retry = Retry(connect=5, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        reqs = session.get(url)
        sleep(3)
        
        soup = BeautifulSoup(reqs.text, 'html.parser')

        # displaying the title
        for title in soup.find_all('title'):
            title2 = title.get_text()
            break
        titlelist.append(title2)
        labellist.append(label)
    data = {'title': titlelist, 'label': labellist}
    return data


for i in np.arange(12):
    # 0 - 367, 368 
    retdf = pd.DataFrame(findTitles(urlsdf['url'].iloc[367*i : 367*i+367],urlsdf['label'].iloc[367*i : 367*i+367] ))
    titlesdf = pd.concat(titlesdf, retdf)


SSLError: HTTPSConnectionPool(host='ble.ac', port=443): Max retries exceeded with url: /1BTa8eN (Caused by SSLError(SSLCertVerificationError("hostname 'ble.ac' doesn't match 'n.ssl-261-default.ssl.fastly.net'")))

# Model

In [36]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [37]:
#read in clickbait data for basic zip files
    #dataset used: https://github.com/bhargaviparanjape/clickbait/tree/master/dataset (2016)
def readin(path):
    with open(path, 'r') as f:
        return [l.strip() for l in f if len(l.strip()) > 0]
        f.close()

#read in as list
clickbait = readin("clickbait_data")
nonclickbait = readin("non_clickbait_data")

#convert to clickbait to df
clickbaitdf = pd.DataFrame(clickbait)
clickbaitdf.columns = ['Titles']
clickbaitdf['isClickbait'] = 1

#convert nonclickbait to df
nonclickbaitdf = pd.DataFrame(nonclickbait)
nonclickbaitdf.columns = ['Titles']
nonclickbaitdf['isClickbait'] = 0

totaldatadf = pd.concat([clickbaitdf, nonclickbaitdf])

In [38]:
#split dataset into target training
X = totaldatadf['Titles']
y = totaldatadf['isClickbait']

In [39]:
#split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [40]:
#convert textual headlines to numerical format
tfidf = TfidfVectorizer()
tfidf.fit(X_train)

TfidfVectorizer()

In [41]:
#create the model
logreg = LogisticRegressionCV()

#fit the model 
logreg.fit(tfidf.transform(X_train), y_train)

/Users/supriyadixit/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/supriyadixit/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

LogisticRegressionCV()

In [42]:
#test accuracy
pipe = make_pipeline(tfidf, logreg)
predictions = logreg.predict(tfidf.transform(X_test))
print("accuracy = ", accuracy_score(y_test, predictions))

accuracy =  0.97453125


In [49]:
#hookup a single url here
import requests
from bs4 import BeautifulSoup
 
# target url
url = 'https://www.buzzfeed.com/peasandpeonies/15-insanely-cheesy-delicious-ways-to-eat-a-wheel-1vim4'
 
# making requests instance
reqs = requests.get(url)
 
# using the BeautifulSoup module
soup = BeautifulSoup(reqs.text, 'html.parser')
 
# displaying the title
print("Title of the website is : ")
for title in soup.find_all('title'):
    title2 = title.get_text()
    print(title2)
    break

Title of the website is : 
15 Insanely Delicious Ways To Eat More Brie


In [50]:
testdata = [title2]
pipe = make_pipeline(tfidf, logreg)
predicted = logreg.predict(tfidf.transform(testdata))
print("Predicted: ", predicted)

Predicted:  [1]
